# Does python use pass-by-reference or pass-by-value?

This is a loaded question because the answer is actually neither. 
In usage it may look like it is sometimes the former and sometimes the latter. But actually python uses something known as call-by-object or call-by-object-reference.

Let's try some examples:

In [1]:
def function1(arg1):
    print (f"arg1 passed in: {arg1}, id(arg1): {id(arg1)}")
    arg1 = arg1 + 1
    print (f"arg1 after modification: {arg1}, id(arg1): {id(arg1)}")

A small detour first. Let's try and understand what the id() function does.
The id() function takes an object as the input and returns the "identity" of the object. The identity is an integer which is unique and constant for this object during its lifetime. For e.g. 

In [2]:
a = 10
print(id(a))

4410156672


In [3]:
a = a + 1
print(id(a))

4410156704


So what happened here? How did the id change?

Because 'a' here is pointing to an immutable object, changing the value of 'a' resulted in 'a' pointing to a new object. 
The same is not the case with mutable objects.

In [4]:
b = list([1, 2, 3])
print (f"b = {b}, id(b) = {id(b)}")

b = [1, 2, 3], id(b) = 4444772872


In [5]:
b.append(4)
print (f"b = {b}, id(b) = {id(b)}")

b = [1, 2, 3, 4], id(b) = 4444772872


Here b still points to the same object.

Now coming back to the point of function arguments. Let's see what happens when we call the function function1.

In [6]:
print (f"a before function call = {a},  id(a) = {id(a)}")
function1(a)
print (f"a after function call = {a}, id(a) = {id(a)}")

a before function call = 11,  id(a) = 4410156704
arg1 passed in: 11, id(arg1): 4410156704
arg1 after modification: 12, id(arg1): 4410156736
a after function call = 11, id(a) = 4410156704


So what has happened here is that initially it passes the object but as soon as it is modified, because it is an immutable object, a new object is created and it is used in the function and outside the scope of the function, the object remains unchanged.

That means that when we pass immutable objects to a function, the passing acts like call-by-value. The object reference is passed to the function parameters. They can't be changed within the function, because they can't be changed at all, i.e. they are immutable.

Now what happens if we pass in mutable objects?

In [7]:
def function2(arg):
    print(f"arg passed in = {arg}, id(arg) = {id(arg)}")
    arg.append(5)
    print(f"arg after modification = {arg}, id(arg) = {id(arg)}")

In [8]:
print(f"b before function call = {b}, id(b) = {id(b)}")
function2(b)
print(f"b after function call = {b}, id(b) = {id(b)}")

b before function call = [1, 2, 3, 4], id(b) = 4444772872
arg passed in = [1, 2, 3, 4], id(arg) = 4444772872
arg after modification = [1, 2, 3, 4, 5], id(arg) = 4444772872
b after function call = [1, 2, 3, 4, 5], id(b) = 4444772872


So there you have it, when passing mutable objects, it acts like pass-by-reference.

## Side effects

A function is said to have a side effect if the function changes the callers environment in other ways than just producing a return value. 
For e.g. modifying a global variable, modifying one of the arguments etc. 

In [9]:
global_var = 10

def function2():
    global global_var
    global_var = 11

function2()
print(global_var)

11


Note: Note here that we have to use the label 'global' before using the global variable. If we don't do this, it will always create a local copy and the global variable will not get changed.

Most of the time the side effects are intended, but in some cases it can lead to trouble.
See the following example.

In [10]:
def function3(x, list=[]):
    for i in range(x):
        list.append(i)
    print(list)
    
    
function3(3)

[0, 1, 2]


In [11]:
function3(3, ['a', 'b', 'c'])


['a', 'b', 'c', 0, 1, 2]


In [12]:
function3(4)

[0, 1, 2, 0, 1, 2, 3]


### Wait? What happened here?

The first two calls print the expected results, but the third call is printing something weird.
Whats happening here is that the first call creates a new list in the memory and appends 0, 1, 2 to this list.
In the second call we have passed a mutable argument(a list) and the function uses this list to work on and hence gets the expected result. 
But in the third call, we are not passing in any argument, and hence the function uses the list created in the first call and uses it and appends the remaining. Hence the weird result.

Here is the same function with some logs added to help understand.

In [13]:
def function3_with_logs(x, list=[]):
    print(f"List at entry = {list}, id(list) = {id(list)}")
    for i in range(x):
        list.append(i)
    print(f"List at exit = {list}, id(list) = {id(list)}")
    
function3_with_logs(3)

List at entry = [], id(list) = 4443403720
List at exit = [0, 1, 2], id(list) = 4443403720


In [14]:
function3_with_logs(3, ['a', 'b', 'c'])

List at entry = ['a', 'b', 'c'], id(list) = 4443664328
List at exit = ['a', 'b', 'c', 0, 1, 2], id(list) = 4443664328


In [15]:
function3_with_logs(4)

List at entry = [0, 1, 2], id(list) = 4443403720
List at exit = [0, 1, 2, 0, 1, 2, 3], id(list) = 4443403720


So there you have it. Hope this makes it clear.

## Passing variable number of arguments

The special syntax \*args and \*\*kwargs can be used to pass a variable number of arguments to a python function. The single asterisk (\*args) is used to pass a non-keyworded, variable-length argument list, and the double asterisk form (\*\*kwargs) is used to pass a keyworded, variable-length argument list.

Note: The identifiers args and kwargs are just a naming convention.

In [16]:
def function_with_variable_args1(*args):
    for arg in args:
        print (f"argument passed : {arg}")
        
function_with_variable_args1(1, 2, 3, 4)

argument passed : 1
argument passed : 2
argument passed : 3
argument passed : 4


In [17]:
def function_with_variable_args2(**kwargs):
    for key in kwargs:
        print(f"argument passed - Key = {key}, value = {kwargs[key]}")

function_with_variable_args2(a='A', b="Boat", c=100, d=3.1415)

argument passed - Key = a, value = A
argument passed - Key = b, value = Boat
argument passed - Key = c, value = 100
argument passed - Key = d, value = 3.1415


These arguments can be combined. When combining, the arguments must be ordered in the following manner.

1. Normal positional arguments
2. \*args
3. keyworded arguments
4. \*\*kwargs

In [18]:
def function_with_variable_args3(a, *args, b="banana", **kwargs):
    print (f"Normal positional argument = {a}")
    for arg in args:
        print(f"args : {arg}")
    print(f"keyworded argument = {b}")
    for key in kwargs:
        print(f"kwargs - key = {key}, value = {kwargs[key]}")
    
function_with_variable_args3(10, 20, "thirty", b="bababananana", c=40, d="fifty", e=60.0)

Normal positional argument = 10
args : 20
args : thirty
keyworded argument = bababananana
kwargs - key = c, value = 40
kwargs - key = d, value = fifty
kwargs - key = e, value = 60.0


### Using \*args and \*\*kwargs in function calls.

We can use \*args and \*\*kwargs in function calls as well in a similar manner.

In [19]:
def function4(arg1, arg2, arg3, arg4):
    print("arguments =", arg1, arg2, arg3, arg4)
    
list1 = [1, 2, 3, 4]
function4(*list1)

arguments = 1 2 3 4


In [20]:
list2 = [3, 4]
function4(1, 2, *list2)

arguments = 1 2 3 4


In [21]:
arguments1 = {"arg1" : 1, "arg2" : "banana", "arg3" : 3, "arg4" : 4.0}

function4(**arguments1)

arguments = 1 banana 3 4.0


In [22]:
arguments2 = {"arg3": 3, "arg4":4.0}
function4(1, "banana", **arguments2)

arguments = 1 banana 3 4.0
